# Nest

Prepares data for the Nest page 

In [1]:
import os
import pandas as pd
from events import (
    load_event_data, filter_public_events, filter_by_project, prepare, save_files, count_wards, filter_signature_events, filter_community_events, load_raw_data
)
import ticket_helpers


Set output directory

In [2]:
OUT_DIR = '../../../../docs/metrics/events/nest/_data/events'
os.makedirs(OUT_DIR, exist_ok=True)

Read in event data and apply Airtable filters.

* Project_type is 'Produce (Signature)'
* Project_name is '10 - Melt'
* Event_type is 'Public Event -' and 'CLE - Community Engagement'

In [3]:
data = (
  load_raw_data()
    .pipe(filter_signature_events)
    .pipe(filter_by_project, '10 - Nest')
  )

Run the processing and save the files

In [4]:
data.pipe(prepare).pipe(save_files, OUT_DIR)

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,status,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size
366,15.0,NaN,242,2023-04-06,R&D Workshop - Young People/Tech,['CLE - Youth Voice session'],10 - Nest: R&D Workshop - Young People/Tech,NaN,['LS26 8DX'],10 - Nest,Produce (Signature),['Season 3 - Dreaming'],2023-04-03,01 - Greenlit,NaN,['Garforth and Swillington'],Garforth and Swillington,E05013830,15.0
476,47.0,NaN,244,2023-04-30,NEST: Acting Intake Auditions,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake Auditions,NaN,NaN,10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-04-29,01 - Greenlit,NaN,NaN,<NA>,NaN,47.0
558,20.0,NaN,245,2023-06-03,NEST: Acting Intake May Course,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake May Course,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-05-30,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,20.0
577,50.0,NaN,1629,2023-06-11,NEST: Protagonist Auditions,['CLE - Skills and Talent session'],10 - Nest: NEST: Protagonist Auditions,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-06-10,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,50.0
698,11.0,NaN,1776,2023-07-06,NEST: Theatre Show Trip,"['CLE - Skills and Talent session', 'CLE - Par...",10 - Nest: NEST: Theatre Show Trip,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-07-06,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,11.0
715,7.0,NaN,243,2023-07-12,NEST: Protagonist Auditions 2nd Round,"['CLE - Skills and Talent session', 'CLE - Par...",10 - Nest: NEST: Protagonist Auditions 2nd Round,NaN,['LS10 1NE'],10 - Nest,Produce (Signature),['Season 1 - Awakening'],2023-07-12,01 - Greenlit,NaN,['Hunslet and Riverside'],Hunslet and Riverside,E05011399,7.0
738,29.0,NaN,625,2023-08-04,NEST: Acting Intake Course Compass Collective,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake Course Compass ...,NaN,['LS2 7BS'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-07-24,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,29.0
739,15.0,NaN,2031,2023-08-04,NEST: Acting Intake July Course 2,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake July Course 2,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),NaN,2023-07-24,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,15.0
752,12.0,NaN,397,2023-08-04,NEST: Acting Intake July Course 1,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake July Course 1,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-07-27,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,12.0
772,86.0,NaN,357,2023-08-13,Melt: Ensemble Rehearsals,['CLE - Youth Voice session'],10 - Nest: Melt: Ensemble Rehearsals,NaN,['LS11 5DJ'],10 - Nest,Produce (Signature),['Season 3 - Dreaming'],2023-08-11,01 - Greenlit,NaN,NaN,<NA>,NaN,86.0


In [5]:
PUBLIC_EVENTS_DIR = OUT_DIR+'/public'
os.makedirs(PUBLIC_EVENTS_DIR, exist_ok=True)


data.pipe(filter_public_events).pipe(prepare).pipe(save_files, PUBLIC_EVENTS_DIR)

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,status,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size
815,1338.0,NaN,241,2023-09-09,Melt (Run),['Public Event - Performance'],10 - Nest: Melt (Run),1542.0,['LS26 8AL'],10 - Nest,Produce (Signature),['Season 3 - Dreaming'],2023-09-05,01 - Greenlit,"ev_2809632, ev_2809633, ev_2809634, ev_28096...",['Garforth and Swillington'],Garforth and Swillington,E05013830,1338.0


In [6]:
COMMUNITY_EVENTS_DIR = OUT_DIR+'/community'
os.makedirs(COMMUNITY_EVENTS_DIR, exist_ok=True)

data.pipe(filter_community_events).pipe(prepare).pipe(save_files, COMMUNITY_EVENTS_DIR)


,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,status,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size
366,15.0,NaN,242,2023-04-06,R&D Workshop - Young People/Tech,['CLE - Youth Voice session'],10 - Nest: R&D Workshop - Young People/Tech,NaN,['LS26 8DX'],10 - Nest,Produce (Signature),['Season 3 - Dreaming'],2023-04-03,01 - Greenlit,NaN,['Garforth and Swillington'],Garforth and Swillington,E05013830,15.0
476,47.0,NaN,244,2023-04-30,NEST: Acting Intake Auditions,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake Auditions,NaN,NaN,10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-04-29,01 - Greenlit,NaN,NaN,<NA>,NaN,47.0
558,20.0,NaN,245,2023-06-03,NEST: Acting Intake May Course,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake May Course,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-05-30,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,20.0
577,50.0,NaN,1629,2023-06-11,NEST: Protagonist Auditions,['CLE - Skills and Talent session'],10 - Nest: NEST: Protagonist Auditions,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-06-10,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,50.0
698,11.0,NaN,1776,2023-07-06,NEST: Theatre Show Trip,"['CLE - Skills and Talent session', 'CLE - Par...",10 - Nest: NEST: Theatre Show Trip,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-07-06,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,11.0
715,7.0,NaN,243,2023-07-12,NEST: Protagonist Auditions 2nd Round,"['CLE - Skills and Talent session', 'CLE - Par...",10 - Nest: NEST: Protagonist Auditions 2nd Round,NaN,['LS10 1NE'],10 - Nest,Produce (Signature),['Season 1 - Awakening'],2023-07-12,01 - Greenlit,NaN,['Hunslet and Riverside'],Hunslet and Riverside,E05011399,7.0
738,29.0,NaN,625,2023-08-04,NEST: Acting Intake Course Compass Collective,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake Course Compass ...,NaN,['LS2 7BS'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-07-24,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,29.0
739,15.0,NaN,2031,2023-08-04,NEST: Acting Intake July Course 2,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake July Course 2,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),NaN,2023-07-24,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,15.0
752,12.0,NaN,397,2023-08-04,NEST: Acting Intake July Course 1,['CLE - Participant / Volunteer thank you even...,10 - Nest: NEST: Acting Intake July Course 1,NaN,['LS2 7UP'],10 - Nest,Produce (Signature),['Season 2 - Playing'],2023-07-27,01 - Greenlit,NaN,['Little London and Woodhouse'],Little London and Woodhouse,E05011403,12.0
772,86.0,NaN,357,2023-08-13,Melt: Ensemble Rehearsals,['CLE - Youth Voice session'],10 - Nest: Melt: Ensemble Rehearsals,NaN,['LS11 5DJ'],10 - Nest,Produce (Signature),['Season 3 - Dreaming'],2023-08-11,01 - Greenlit,NaN,NaN,<NA>,NaN,86.0


## Ticket processing

Extract the ticket data

In [7]:
ticket_sales = data.pipe(filter_public_events).pipe(ticket_helpers.get_tickets_for_event)

By Ward

In [8]:
ticket_sales.pipe(ticket_helpers.summarise_by_ward).reset_index().to_csv(
    os.path.join(OUT_DIR, 'tickets_by_ward.csv'), index=False)


By Date

In [9]:
(
  ticket_sales
    .pipe(ticket_helpers.summarise_by_date)
    .resample('W-FRI').sum()
    .cumsum().to_csv(os.path.join(OUT_DIR, 'tickets_by_date.csv'))
)